In [1]:
# Normalise features
# Find the optimal parameters C (inverse regulalisation strengths)
# Find the best features to predict stock movement (up or down)
# Convert probability into absolute values (stock % increase/decrease)
# SVC, RandomForest, MinMax, LongShortTermMemory

%matplotlib inline
%pylab inline

# Nice Formatting within Jupyter Notebook
%matplotlib inline
from IPython.display import display # Allows multiple displays from a single code-cell
from jupyterthemes import jtplot

#import classes
from company import Company
from onestep_baseline_company import OneStepBaselineCompany
from onestep_lstm_company import OneStepLSTMCompany
from multistep_baseline_company import MultiStepBaselineCompany
from multistep_lstm_company import MultiStepLSTMCompany

jtplot.style(theme='grade3')
jtplot.style(theme='grade3')
jtplot.style(theme='grade3')


Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In [ ]:
from multistep_lstm_company import MultiStepLSTMCompany
from alpha_vantage.techindicators import TechIndicators
from time import time
import pandas as pd
from sklearn.preprocessing import MinMaxScaler



class MultiStepLSTMCompanyTechIndicators(MultiStepLSTMCompany):
    def __init__(self, name, train_start_date_string, train_end_test_start_date_string, test_end_date_string,
                 n_lag, n_seq, n_epochs, n_batch, n_neurons, tech_indicators=[]):
        self.tech_indicators = TechIndicators(key='3OMS720IM6CRC3SV', output_format='pandas')
        self.all_tech_indicators = ["ad", "adosc", "adx", "adxr", "apo", "aroon", "aroonosc", 
                                    "bbands", "bop", "cci", "cmo", "dema", "dx", "ema", "ht_dcperiod", 
                                    "ht_dcphase", "ht_phasor", "ht_sine", "ht_trendline", "ht_trendmode", 
                                    "kama", "macd", "macdext", "mama", "mfi", "midpoint", "midprice", 
                                    "minus_di", "minus_dm", "mom", "natr", "obv", "plus_di", "plus_dm", 
                                    "ppo", "roc", "rocr", "rsi", "sar", "sma", "stoch", "stochf", "stochrsi", 
                                    "t3", "tema", "trange", "trima", "trix", "ultsoc", "willr", "wma"]
        if tech_indicators == "all":
            self.input_tech_indicators_list = self.all_tech_indicators
        else:
            self.input_tech_indicators_list = tech_indicators
        self.n_indicators = len(tech_indicators)
        #self.all_tech_indicators
        MultiStepLSTMCompany.__init__(self, name, train_start_date_string, train_end_test_start_date_string, test_end_date_string,
                 n_lag, n_seq, n_epochs, n_batch, n_neurons)
        
    
    def add_tech_indicators_dataframe(self, price_series, indicators):
        combined = price_series
        for ind in indicators:
            print("ind", ind)
            while True: # try again until success
                try:
                    ind_series = self.get_indicator(ind, self.train_start_date_string, self.test_end_date_string)
                    combined = pd.concat([combined, ind_series], axis=1)
                    break
                except:
                    print("Retrying to download indicator ", ind)
                    pass
        
        return combined

    def get_indicator(self, ind_name, start, end):
        data, meta_data = getattr(self.tech_indicators, "get_" + ind_name)(self.name, interval="daily")
        data.index = pd.to_datetime(data.index)
        data = self.get_filtered_series(data, start, end)
        return data
    
    def preprocess_data(self):
        price_series = self.train_raw_series.append(self.test_raw_series)
        display("price data series", len(price_series), price_series)
        if len(self.input_tech_indicators_list) > 0:
            #add additional technical indicators
            combined = self.add_tech_indicators_dataframe(price_series, self.input_tech_indicators_list)
        else:
            combined = price_series
        
        display("combined", combined)
            
        supervised_pd = self.timeseries_to_supervised(combined, self.n_lag, self.n_seq)
        display("supervised", supervised_pd)
        
        cutoff = len(self.train_raw_series) - self.n_seq - 1
        train_supervised_values = supervised_pd.values[:cutoff - self.n_lag + 1]
        display("train supervised values", train_supervised_values)
        test_supervised_values = supervised_pd.values[cutoff + self.n_seq - self.n_lag:]
        display("test supervised values", test_supervised_values)

        self.scaler, scaled_train_supervised, scaled_test_supervised = self.scale(train_supervised_values,
                                                                                  test_supervised_values)
        # delete unnecessary variables for prediction except price (should be var1)
        display("scaled train supervised", scaled_train_supervised)
        display("scaled test supervised", scaled_test_supervised)
        
        return scaled_train_supervised, scaled_test_supervised

    # evaluate the persistence model
    def predict(self):
        self.reset()
        # walk-forward validation on the test data
        predictions = pd.Series()
        # Index is datetime
        test_index = self.test_raw_series.index
        for i in range(len(self.test_scaled)):
            # make multi-step forecast
            X, y = self.test_scaled[i, 0:self.n_lag+self.n_indicators], self.test_scaled[i, self.n_lag+self.n_indicators:]
            print("X: ", X, "y: ", y)
            pred = self.forecast_lstm(X)
            print("Prediction: ", pred)
            # store forecast
            predictions.at[test_index[i]] = pred

        # display("predictions before inverse transform", predictions)
        # inverse transform
        predictions = self.inverse_transform(self.train_raw_series.append(self.test_raw_series), predictions,
                                             len(self.test_raw_series))
        print("Predictions after inverse transform")
        display(predictions)
        return predictions

    # scale train and test data to [-1, 1]
    def scale(self, train_raw, test_raw):
        # fit scaler with 1 Dimensional array data
        scaler = MinMaxScaler(feature_range=(-1, 1))
        # display("fit scaler with train data", scaler_train_data)
        scaler = scaler.fit(train_raw)
        # transform train
        train_scaled = scaler.transform(train_raw)
        # display("train_scaled", train_scaled)
        # transform test
        test_scaled = scaler.transform(test_raw)
        # display("test_scaled", test_scaled)

        return scaler, train_scaled, test_scaled
    
    # fit an LSTM network to training data
    def fit_lstm(self, train):
        # reshape training into [samples, timesteps, features]
        X, y = train[:, 0:self.n_lag+self.n_indicators], train[:, self.n_lag+self.n_indicators:]
        X = X.reshape(X.shape[0], 1, X.shape[1])
        display("train X data", X)
        display("train y data", y)
        # design network
        model = Sequential()
        model.add(LSTM(self.n_neurons, batch_input_shape=(self.n_batch, X.shape[1], X.shape[2]), stateful=True))
        model.add(Dense(y.shape[1]))
        model.compile(loss='mean_squared_error', optimizer='adam')
        # fit network
        for i in range(self.n_epochs):
            model.fit(X, y, epochs=1, batch_size=self.n_batch, verbose=0, shuffle=False)
            model.reset_states()
        return model
    


In [44]:
start_train_date = "01/01/2018"
end_train_start_test_date = "20/01/2018"
end_test_date = "01/02/2018"

multi_step_lstm_tech_indicator = MultiStepLSTMCompanyTechIndicators("MU", start_train_date, end_train_start_test_date, end_test_date, 
                         n_lag=2, n_seq=1, n_epochs=3000, n_batch=1, n_neurons=4, tech_indicators="all")

multi_step_lstm_tech_indicator.train()

'price data series'

22

date
2018-01-02    43.67
2018-01-03    44.98
2018-01-04    46.88
2018-01-05    45.80
2018-01-08    45.55
2018-01-09    42.97
2018-01-10    43.31
2018-01-11    42.82
2018-01-12    42.81
2018-01-16    42.92
2018-01-17    44.26
2018-01-18    43.99
2018-01-19    42.75
2018-01-22    42.88
2018-01-23    43.95
2018-01-24    43.08
2018-01-25    43.01
2018-01-26    43.67
2018-01-29    43.29
2018-01-30    41.67
2018-01-31    43.72
2018-02-01    42.49
Name: 5. adjusted close, dtype: float64

ind ad
ind adosc
ind adx
ind adxr
ind apo
ind aroon
ind aroonosc
ind bbands
ind bop
ind cci
ind cmo
ind dema
ind dx
ind ema
ind ht_dcperiod
ind ht_dcphase
ind ht_phasor
ind ht_sine
ind ht_trendline
ind ht_trendmode
ind kama
ind macd
ind macdext
ind mama
ind mfi
ind midpoint
ind midprice
ind minus_di
ind minus_dm
ind mom
ind natr
ind obv
ind plus_di
ind plus_dm
ind ppo
ind roc
ind rocr
ind rsi
ind sar
ind sma
ind stoch
ind stochf
ind stochrsi
ind t3
ind tema
ind trange
ind trima
ind trix
ind ultsoc
ind willr
ind wma


'combined'

...

'train X data'

array([[[-0.57739558, -1.        , -1.        ,  1.        ,
          1.        ]],

       [[ 0.06633907, -0.11499175, -0.14716474,  0.74566929,
          0.84016113]],

       [[ 1.        ,  1.        ,  0.89501928,  0.74871391,
          0.77214689]],

       [[ 0.46928747,  0.62356196,  1.        ,  0.75165354,
          0.61295544]],

       [[ 0.34643735,  0.68826338,  0.98413234,  0.67401575,
          0.35791102]],

       [[-0.92137592, -0.60107399,  0.08509241,  0.26362205,
          0.02672373]],

       [[-0.75429975,  0.59338447,  0.43850134,  0.01102362,
         -0.21105636]],

       [[-0.995086  , -0.06500125,  0.12930555, -0.32099738,
         -0.48311333]],

       [[-1.        , -0.46997936, -0.26654422, -0.63643045,
         -0.73697083]],

       [[-0.94594595, -0.74523146, -0.58184146, -1.        ,
         -1.        ]]])

'train y data'

array([[-1.        , -0.75      , -0.86666667, ...,  0.80032144,
         1.        , -1.        ],
       [-0.28522757, -1.        , -1.        , ...,  0.95401349,
         0.46675237,  0.0990099 ],
       [ 0.44561894,  1.        , -1.        , ...,  1.        ,
         0.34411327,  1.        ],
       ...,
       [-0.07940945, -0.25      ,  0.6       , ..., -1.        ,
        -0.94603734, -0.33843384],
       [-0.10544247, -0.5       ,  0.46666667, ..., -0.26668327,
        -0.28869751, -0.09315932],
       [-0.1314755 , -0.75      ,  0.33333333, ..., -0.55327553,
        -0.42114579, -0.00360036]])

NameError: name 'Sequential' is not defined

In [26]:
#multi_step_lstm_tech_indicator.tech_indicators.__dict__.keys()
print(multi_step_lstm_tech_indicator.all_tech_indicators)
#data = multi_step_lstm_tech_indicator.get_rsi()
#X, y = multi_step_lstm_tech_indicator.train_scaled[:, 0:multi_step_lstm_tech_indicator.n_lag], multi_step_lstm_tech_indicator.train_scaled[:, multi_step_lstm_tech_indicator.n_lag:]
#X = X.reshape(X.shape[0], 1, X.shape[1])

['ad', 'adosc', 'adx', 'adxr', 'apo', 'aroon', 'aroonosc', 'bbands', 'bop', 'cci', 'cmo', 'dema', 'dx', 'ema', 'ht_dcperiod', 'ht_dcphase', 'ht_phasor', 'ht_sine', 'ht_trendline', 'ht_trendmode', 'kama', 'macd', 'macdext', 'mama', 'mfi', 'midpoint', 'midprice', 'minus_di', 'minus_dm', 'mom', 'natr', 'obv', 'plus_di', 'plus_dm', 'ppo', 'roc', 'rocr', 'rsi', 'sar', 'sma', 'stoch', 'stochf', 'stochrsi', 't3', 'tema', 'trange', 'trima', 'trix', 'ultsoc', 'willr', 'wma']


In [ ]:
#import numpy as np
#import pandas as pd
display(multi_step_lstm_tech_indicator)

display("size", len(data.values), " values:", data.values[1:-1])
display("size", len(X), " values:", X)
display(np.concatenate((X, data.values[1:-1]), axis=1))
#display(pd.concat([price, data], axis=1))